---
title: "Lecture 6: Conditioning of linear systems, Matrix structure"
author: "Jamie Haddock"
format: 
    revealjs:
        output-file: Lecture6_slides
        slide-number: true
        chalkboard: 
            buttons: false
        preview-links: auto
        logo: figs/hmc.png
        css: input/slides.css
        incremental: true
        smaller: true
        code-fold: true
    html: 
        code-fold: true
    pdf:
        documentclass: article
        toc: true
        number-sections: true
        geometry:
          - top=1in
          - left=1in
          - bottom=1in
          - right=1in
format-links: false
jupyter: julia-1.9
filters: 
  - input/remove-pause.lua
execute:
  echo: true
  eval: true
---

## Conditioning of Linear Systems

We consider now the conditioning of solving the square linear system $\mathbf{A}\mathbf{x} = \mathbf{b}$.  Here, the data is $\mathbf{A}$ and $\mathbf{b}$, and the solution is $\mathbf{x}$.

. . .

For simplicity, we'll imagine that there are perturbations only to $\mathbf{b}$, while $\mathbf{A}$ is fixed.  Suppose $\mathbf{A}\mathbf{x} = \mathbf{b}$ is perturbed to $$\mathbf{A}(\mathbf{x} + \mathbf{h}) = \mathbf{b} + \mathbf{d}.$$  

. . .

The condition number is the relative change in the solution divided by the relative change in the data, $$\frac{\frac{\|\mathbf{h}\|}{\|\mathbf{x}\|}}{\frac{\|\mathbf{d}\|}{\|\mathbf{b}\|}} = \frac{\|\mathbf{h}\| \|\mathbf{b}\|}{\|\mathbf{d}\|\|\mathbf{x}\|}.$$

---

Since $\mathbf{h} = \mathbf{A}^{-1}\mathbf{d}$, we can bound $\|\mathbf{h}\|$ as $$\|\mathbf{h}\| \le \|\mathbf{A}^{-1}\|\|\mathbf{d}\|.$$

. . .

Similarly, we have $\|\mathbf{b}\| \le \|\mathbf{A}\| \|\mathbf{x}$ and so $$\frac{\|\mathbf{h}\| \|\mathbf{b}\|}{\|\mathbf{d}\|\|\mathbf{x}\|} \le \frac{\|\mathbf{A}^{-1}\|\|\mathbf{d}\|\|\mathbf{A}\|\|\mathbf{x}\|}{\|\mathbf{d}\|\|\mathbf{x}\|} = \|\mathbf{A}^{-1}\|\|\mathbf{A}\|.$$

. . .

This bound is tight -- the inequalities are equations for some choices of $\mathbf{b}$ and $\mathbf{d}$.

::: {.callout-note icon=false}
## Definition: Matrix condition number

The **matrix condition number** of an invertible square matrix $\mathbf{A}$ is $$\kappa(\mathbf{A}) = \|\mathbf{A}^{-1}\|\|\mathbf{A}\|.$$  This value depends on the choice of norm; a subscript on $\kappa$ such as 1, 2, or $\infty$ is used if clarification is needed.  If $\mathbf{A}$ is singular, we define $\kappa(\mathbf{A}) = \infty$.  
:::

---

::: {.callout-warning icon=false}
## Theorem: Conditioning of linear systems
If $\mathbf{A}(\mathbf{x} + \triangle \mathbf{x}) = \mathbf{b} + \triangle \mathbf{b}$, then $$\frac{\|\triangle \mathbf{x}\|}{\|\mathbf{x}\|} \le \kappa(\mathbf{A}) \frac{\|\triangle \mathbf{b}\|}{\|\mathbf{b}\|}.$$

If $(\mathbf{A} + \triangle \mathbf{A})(\mathbf{x} + \triangle \mathbf{x}) = \mathbf{b}$, then $$\frac{\|\triangle \mathbf{x}\|}{\|\mathbf{x}\|} \le \kappa(\mathbf{A}) \frac{\|\triangle \mathbf{A}\|}{\|\mathbf{A}\|},$$
in the limit $\|\triangle \mathbf{A}\| \rightarrow 0$.
:::

. . .

::: {.callout-caution icon=false}
## Exercise: Lower bound on condition number
Show that $\kappa(\mathbf{A}) \ge 1$.
:::
<details><summary>Answer:</summary> 
We have $1 = \|\mathbf{I}\| = \|\mathbf{A}\mathbf{A}^{-1}\| \le \|\mathbf{A}\|\|\mathbf{A}^{-1}\| = \kappa(\mathbf{A}).$
</details>

[A condition number equal to 1 is the best we can hope for -- this means the relative perturbation in the solution is the same size as that of the data.  If a matrix has condition number $10^t$ indicates that in floating-point arithmetic, roughly $t$ digits are lost in computing the solution $\mathbf{x}$.  If $\kappa(\mathbf{A})> 1/\epsilon_{\text{mach}}$, then for numerical purposes, the matrix $\mathbf{A}$ is effectively singular.  ]{.content-hidden when-format='revealjs' when-format='pptx'} 

---

[Julia has the function `cond` to compute the matrix condition number.  The $\ell_2$ norm is used by default in this calculation.]{.content-hidden when-format='revealjs' when-format='pptx'} We'll begin with an example of a *Hilbert matrix* which is famously ill-conditioned.

In [3]:
using LinearAlgebra

A = [ 1/(i+j) for i in 1:6, j in 1:6 ]
κ = cond(A)

5.109816297946132e7

When solving a linear system with this matrix, we will lose nearly 8 digits of accuracy due to the ill-conditioning of this problem!

In [4]:
x = 1:6
b = A*x;

We perturb the system randomly by $10^{-10}$ in norm.

In [5]:
◬A = randn(size(A)); ◬A = 1e-10*(◬A/opnorm(◬A));
◬b = randn(size(b)); ◬b = 1e-10*normalize(◬b);

---

We solve the perturbed problem and see how the solution is changled.

In [8]:
new_x = ((A + ◬A) \ (b+◬b))
◬x = new_x - x

6-element Vector{Float64}:
  1.1053746015177168e-5
 -0.00017684631114378568
  0.0008826710483242906
 -0.001887105940777456
  0.0018119922449173487
 -0.0006427654517580095

In [9]:
@show relative_error = norm(◬x) / norm(x);

relative_error = norm(◬x) / norm(x) = 0.0002977597146152902


In [11]:
println("Upper bound due to b: $(κ*norm(◬b)/norm(b))")
println("Upper bound due to A: $(κ*norm(◬A)/norm(A))")

Upper bound due to b: 0.0006723667714371329
Upper bound due to A: 0.006609488624373632


---

These errors are due to our manual perturbations we made to the data.  Even just machine roundoff perturbs this data and affects the solution of this ill-conditioned problem.  This error will scale with $\epsilon_{\text{mach}}$.

In [12]:
◬x = A\b - x
@show relative_error = norm(◬x)/norm(x);
@show rounding_bound = κ*eps();

relative_error = norm(◬x) / norm(x) = 7.822650774976615e-10
rounding_bound = κ * eps() = 1.134607141116935e-8


---

Larger Hilbert matrices are even more ill-conditioned and their linear systems suffer from more error during solution.

In [16]:
A = [ 1/(i+j) for i=1:14, j=1:14 ];
κ = cond(A)                          #exceeds 1/eps()

5.802584125151949e17

In [17]:
rounding_bound = κ*eps()

128.8432499613623

In [18]:
x = 1:14
b = A*x
◬x = A\b - x
@show relative_error = norm(◬x)/norm(x);

relative_error = norm(◬x) / norm(x) = 4.469466154206132


. . .

There are zero accurate digits!

## Residual and backward error

When we don't know the solution of a linear system, we cannot compare our approximate computed solution to the true solution, so we use the residual error.

::: {.callout-note icon=false}
## Definition: Residual of a linear system
For the problem $\mathbf{A}\mathbf{x} = \mathbf{b}$, the **residual** at a solution estimate $\hat{\mathbf{x}}$ is $$\mathbf{r} = \mathbf{b} - \mathbf{A}\hat{\mathbf{x}}.$$
:::

. . . 

A zero residual means we have an exact solution, and if the matrix is rank $n$, then we have $\hat{\mathbf{x}} = \mathbf{x}$.  

. . .

More generally, though, we have $$\mathbf{A}\hat{\mathbf{x}} = \mathbf{b} - \mathbf{r}.$$  This means that $\hat{\mathbf{x}}$ is an exact solution for a linear system with right hand error changed by $-\mathbf{r}$.  

. . .

This is what we search for when studying background error!

---

Hence, residual error of a linear system is the system's backward error.  We can connect this error to the forward error by making the definition $\mathbf{h} = \hat{\mathbf{x}} - \mathbf{x}$ in the equation $\mathbf{A}(\mathbf{x}+\mathbf{h}) = \mathbf{b}+\mathbf{d}$.

. . .

Then $$\mathbf{d} = \mathbf{A}(\mathbf{x}+\mathbf{h}) - \mathbf{b} = \mathbf{A}\mathbf{h} = -\mathbf{r}.$$

. . .

Thus, our previous theorem yields $$\frac{\|\mathbf{x} - \hat{\mathbf{x}}\|}{\|\mathbf{x}\|} \le \kappa(\mathbf{A}) \frac{\|\mathbf{r}\|}{\|\mathbf{b}\|}.$$

[The relationship between relative error and the relative residual is scaling by the matrix condition number.]{.content-hidden when-format='revealjs' when-format='pptx'} 

. . .

::: {.callout-warning icon=false}
## Fact: 
When solving a linear system, we can only expect that the backward (residual) error is small, not the error, since this will suffer from scaling by the matrix condition number.
:::

# Matrix structure

Many matrices typically encountered in scientific computing have special structure.  It can be *very* helpful to understand and exploit these special structures!

## Diagonal dominance

An $n \times n$ matrix $\mathbf{A}$ is **(row) diagonally dominant** if $$|A_{ii}| > \sum_{j=1//j\not=i}^n |A_{ij}| \text{ for each } i=1, \cdots, n.$$ [This says that the magnitude of entries on the diagonal are larger than the sum of magnitudes of entries in the same row off-diagonal.]{.content-hidden when-format='revealjs' when-format='pptx'} 

. . .

* Diagonally dominant matrices are guaranteed to be invertible.
* Diagonally dominant matrices do not need row-pivoting for elimination/LU stability.


## Banded matrices

::: {.callout-note icon=false}
## Definition: Bandwidth
A matrix $\mathbf{A}$ has **upper bandwidth** $b_u$ if $j - i > b_u$ implies $A_{ij} = 0$, and **lower bandwidth** $b_l$ if $i-j > b_l$ implies $A_{ij} = 0$.  We say the total **bandwidth** is $b_u + b_l + 1$.  When $b_u = b_l = 1$, we have the important case of a **tridiagonal matrix**.
:::

In [19]:
A = diagm( -1=>[4,3,2,1,0], 0=>[2,2,0,2,1,2], 1=>fill(-1,5) )

6×6 Matrix{Int64}:
 2  -1   0   0   0   0
 4   2  -1   0   0   0
 0   3   0  -1   0   0
 0   0   2   2  -1   0
 0   0   0   1   1  -1
 0   0   0   0   0   2

---

In [21]:
using FundamentalsNumericalComputation

L,U = FNC.lufact(A)
L

6×6 LowerTriangular{Float64, Matrix{Float64}}:
 1.0   ⋅     ⋅        ⋅         ⋅    ⋅ 
 2.0  1.0    ⋅        ⋅         ⋅    ⋅ 
 0.0  0.75  1.0       ⋅         ⋅    ⋅ 
 0.0  0.0   2.66667  1.0        ⋅    ⋅ 
 0.0  0.0   0.0      0.214286  1.0   ⋅ 
 0.0  0.0   0.0      0.0       0.0  1.0

In [22]:
U

6×6 UpperTriangular{Float64, Matrix{Float64}}:
 2.0  -1.0   0.0    0.0       0.0       0.0
  ⋅    4.0  -1.0    0.0       0.0       0.0
  ⋅     ⋅    0.75  -1.0       0.0       0.0
  ⋅     ⋅     ⋅     4.66667  -1.0       0.0
  ⋅     ⋅     ⋅      ⋅        1.21429  -1.0
  ⋅     ⋅     ⋅      ⋅         ⋅        2.0

The LU factors are also banded!

::: {.callout-tip icon=false}
## Note: 
The number of flops needed by LU factorization without pivoting is $\mathcal{O}(b_u b_t n)$ when the upper and lower bandwidths are $b_u$ and $b_l$.
:::

---

In order for Julia to take advantage of banded matrix advantages if we use an ordinary (dense) matrix representation (since it doesn't know in advance where the zeros are).

In [23]:
n = 10000
A = diagm(0=>1:n, 1=>n-1:-1:1, -1=>ones(n-1))
lu(rand(3,3)) #throwaway to force compilation
@time lu(A);

  3.737181 seconds (7 allocations: 763.016 MiB, 0.76% gc time)


. . .

If we use a sparse matrix representation, the speedup is dramatic!

In [24]:
A = spdiagm(0=>1:n, 1=>n-1:-1:1, -1=>ones(n-1))
lu(A); #throwaway for sparse compile
@time lu(A);

  0.002840 seconds (86 allocations: 9.920 MiB)


## Sparse matrices



<!--
[verbose test]{.content-hidden when-format="revealjs" when-format="pptx"}

::: {.callout-caution icon=false}
## Exercise: 

:::

<details><summary>Answer:</summary> </details>


::: {.callout-note icon=false}
## Definition: 
 
:::


::: {.callout-tip icon=false}
## Note: 
 
:::
-->